In [71]:
import os
import csv
import json
from multiprocessing import Pool
from io import StringIO

Fields in json object

Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name

In [72]:
basefolder = "/home/ubuntu/data/tmp/de/"
filename = "Womens Clothing E-Commerce Reviews.json"
filepath = os.path.join(basefolder, filename)

In [73]:
class ChunkedReader(object):
    
    def __init__(self, filepath, number_of_chunks):
        self.number_of_chunks = number_of_chunks
        self.filepath = filepath

    def read_block_lines(self, block):
        """
        
        """
        assert 0 <= block and block < self.number_of_chunks
        assert 0 < self.number_of_chunks

        with open(self.filepath,"r+") as filehandle:
            filehandle.seek(0,2)
            file_size = filehandle.tell()

            ini = int(file_size * block / self.number_of_chunks)
            end = int(file_size * (1 + block) / self.number_of_chunks)

            if ini <= 0:
                filehandle.seek(0)
            else:
                #ini can be in the middle of the line: read line first
                filehandle.seek(ini-1) 
                filehandle.readline()

            while filehandle.tell() < end:
                line = filehandle.readline()
                yield json.loads(line)

In [77]:
number_of_chunks = 20
number_of_processes = 4

def process_chunk(args):
    reader, block = args
    counter = 0
    age_sum = 0
    for js in reader.read_block_lines(block):
        try:
            age = int(js["Age"])
        except ValueError:
            print(js)
            continue
            
        counter += 1
        age_sum += age
        
    return age_sum, counter

fin = ChunkedReader(filepath, number_of_chunks)
pool = Pool(number_of_processes)
age_super_sum = 0
super_count = 0
for age_sum, count in pool.imap_unordered(process_chunk,zip([fin]*number_of_chunks,range(number_of_chunks))):
    print(f"block: {age_sum} and {count}")
    super_count += count
    age_super_sum += age_sum
average_age = age_super_sum/super_count
print(f"average age {average_age}")

block: 50617 and 1168
block: 51491 and 1178
block: 50797 and 1171
block: 51000 and 1180
block: 51382 and 1186
block: 51708 and 1187
block: 51063 and 1195
block: 50711 and 1178
block: 50789 and 1175
block: 50886 and 1187
block: 51057 and 1173
block: 48668 and 1149
block: 51986 and 1187
block: 50832 and 1182
block: 50968 and 1159
block: 50394 and 1157
block: 49007 and 1151
block: 50417 and 1184
block: 50166 and 1152
block: 50622 and 1187
average age 43.198543813335604
